In [1]:
import os
from transformers import AutoTokenizer
import transformers
import torch

In [2]:
os.environ['CUDA_VISIBLE_DEVICES'] = '4'

In [ ]:
# os.environ["HF_HOME"] = os.path.expanduser("~/.huggingface")
# os.environ["HF_HOME"] = "/raid/mridul/nlp"  # Replace with the directory where you want to store Hugging Face datasets and models
# os.environ["HF_HOME"] = "/path/to/api/token"  # Replace with your Hugging Face API token

In [ ]:
model = "meta-llama/Llama-2-7b-chat-hf"

tokenizer = AutoTokenizer.from_pretrained(model)
# pipeline = transformers.pipeline(
#     "text-generation",
#     model=model,
#     torch_dtype=torch.float16,
#     device_map="auto",
# )

In [ ]:

sequences = pipeline(
    'I liked "Breaking Bad" and "Band of Brothers". Do you have any recommendations of other shows I might like?\n',
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    max_length=200,
)
for seq in sequences:
    print(f"Result: {seq['generated_text']}")

In [ ]:
!pip install sentencepiece

In [3]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

tokenizer = T5Tokenizer.from_pretrained("t5-large")
model = T5ForConditionalGeneration.from_pretrained("t5-large")

input_ids = tokenizer("The <extra_id_0> walks in <extra_id_1> park", return_tensors="pt").input_ids
labels = tokenizer("<extra_id_0> cute dog <extra_id_1> the <extra_id_2>", return_tensors="pt").input_ids

# the forward function automatically creates the correct decoder_input_ids
loss = model(input_ids=input_ids, labels=labels).loss
loss.item()

/home/mridul/anaconda3/envs/hgnn/lib/python3.9/site-packages/transformers/models/t5/tokenization_t5.py:238: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-large automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. 

2.295056104660034

In [7]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

tokenizer = T5Tokenizer.from_pretrained("t5-small")
model = T5ForConditionalGeneration.from_pretrained("t5-small")

task_prefix = "translate English to German: "
# use different length sentences to test batching
sentences = ["The house is wonderful.", "I like to work in NYC."]

inputs = tokenizer([task_prefix + sentence for sentence in sentences], return_tensors="pt", padding=True)

output_sequences = model.generate(
    input_ids=inputs["input_ids"],
    attention_mask=inputs["attention_mask"],
    do_sample=False,  # disable sampling to test if batching affects output
)

print(tokenizer.batch_decode(output_sequences, skip_special_tokens=True))

['Das Haus ist wunderbar.', 'Ich arbeite gerne in NYC.']


In [8]:
[task_prefix + sentence for sentence in sentences]

['translate English to German: The house is wonderful.',
 'translate English to German: I like to work in NYC.']

In [9]:
data = [{"news": "(Oct 5, 2011  12:11 PM CDT) Police are still hunting for an  armed and dangerous  man who shot and killed three people and wounded five others at the California quarry where he worked this morning. He also shot and injured another woman in an attempted carjacking later. The suspect is identified as 45-year-old Shareef Allman. The San Jose Mercury News says he moonlighted as producer of a show called Real 2 Real on something called CreaTV in the area. (He interviews Jesse Jackson in one YouTube clip, which is in the gallery. Allman, who spread a message of non-violence, also has written a book called Amazing Grace about female victims of domestic violence, notes the Mercury News. The shooting spree occurred about 4:30am local time when a man left a safety meeting at the Lehigh Southwest Cement Permanente Plant, then returned with a rifle and a handgun, according to police. He fled on foot, and a massive manhunt is under way in the San Jose area. Schools are on lockdown, notes AP.", 
        "masked headline": "____rd Victim Dead in Quarry Shooting; Manhunt Still On", 
        "calculation": "Trans(three)", 
        "ans": "3"}, 
        {"news": "(Mar 4, 2014  11:30 AM) The New York Times followed the Best Picture win for 12 Years a Slave by tweeting a link to an 1853 article on Solomon Northup, the film's historical subject. The original article's lead describes what is to follow as  a more complete and authentic record than has yet appeared  on Northup's kidnapping. Authentic, perhaps, but erroneous: It spelled Northup's name wrong in two different ways. A Twitter user caught the errors, and now the Times has corrected them\u2014just 161 years late, Poynter reports. The article spelled Northup's last name as  Northrop,  while the headline spelled it  Northrup.", 
         "masked headline": "NYT Corrects 1853 Piece After ____ Years Win", 
         "calculation": "Copy(12)", 
         "ans": "12"}, 
        {"news": "(Nov 4, 2008  3:19 PM) Stocks rallied on Election Day as investors applauded the looming conclusion to the presidential race, the Wall Street Journal reports. Continuing declines in interbank lending, and strong third-quarter earnings reports also fueled optimism, with the Dow rising 305.45 to close at 9,625.28. The Nasdaq climbed 53.79 closing at 1,780.12, while the S&P 500 gained 39.45 to settle at 1,005.75. The three-month US Libor dropped 0.10% to 2.706%, its 17th consecutive daily decline, signaling easing lending costs. Archer Daniels Midland rose 14.83% after the agricultural firm reported a doubling in third-quarter profit. MasterCard also shot up 18.2% despite reporting a quarterly loss of $194 million, because the loss was due to an antitrust penalty.", 
         "masked headline": "Stocks Up ____ in Election Rally", 
         "calculation": "Round(305.45,0)", 
         "ans": "305"}
       ]

In [10]:
prompt = 'Read the news article and fill in the blanks at the end of the article \n ' + data[0]['news'] + ' \n Fill in the blanks:  ' + data[0]['masked headline']
prompt

'Read the news article and fill in the blanks at the end of the article \n (Oct 5, 2011  12:11 PM CDT) Police are still hunting for an  armed and dangerous  man who shot and killed three people and wounded five others at the California quarry where he worked this morning. He also shot and injured another woman in an attempted carjacking later. The suspect is identified as 45-year-old Shareef Allman. The San Jose Mercury News says he moonlighted as producer of a show called Real 2 Real on something called CreaTV in the area. (He interviews Jesse Jackson in one YouTube clip, which is in the gallery. Allman, who spread a message of non-violence, also has written a book called Amazing Grace about female victims of domestic violence, notes the Mercury News. The shooting spree occurred about 4:30am local time when a man left a safety meeting at the Lehigh Southwest Cement Permanente Plant, then returned with a rifle and a handgun, according to police. He fled on foot, and a massive manhunt i

In [ ]:
data[0]['masked headline']

In [11]:
inputs = tokenizer(prompt, return_tensors="pt", padding=True)

output_sequences = model.generate(
    input_ids=inputs["input_ids"],
    attention_mask=inputs["attention_mask"],
    do_sample=False,  # disable sampling to test if batching affects output
)

print(tokenizer.batch_decode(output_sequences, skip_special_tokens=True))

['the shooting spree occurred about 4:30am local time. Police are still hunting for']
